# Stream CPU Utilization Data Using Model-driven Telemetry
##### <sup>Authored by Veena Manuel (veedas@cisco.com)</sup>

The use case illustrates how, with the Dial-out mode, you can use telemetry data to proactively monitor CPU
utilization. Monitoring CPU utilization ensures efficient storage capabilities in your network. In a Dial-out mode, the router dials out to the receiver to establish a subscription-based telemetry session.

The following image represents a topology with 4 routers; router P1 is connected to a collector to stream telemetry data.

![Topology](Telemetry-topology.png)

Telemetry involves the following workflow:
- Define: You define a subscription to stream data from the router to the receiver. To define a subscription,
you create a destination-group and a sensor-group.
- Deploy: The router establishes a subscription-based telemetry session and streams data to the receiver.
You verify subscription deployment on the router.
- Operate: You consume and analyse telemetry data using open-source tools, and take necessary actions
based on the analysis.

To gain an indepth understanding about telemetry streaming modes, see the [Telemetry Configuration Guide](https://www.cisco.com/c/en/us/support/routers/8000-series-routers/products-installation-and-configuration-guides-list.html).

## Configuration steps
* [Bring up emulator session with the base network and configurations](#step1)
* [Configure a Telemetry Session Between the Router and the Collector](#step2)
* [Verify the Telemetry configuration](#step3)
* [Operate on Telemetry Data](#step4)

## <a name="step1"></a>Bring up emulator session with the base network and configurations

In order to set up the emulator, you have to execute the cell below by clicking the play button at the top of this notebook. 

In the cell below, the python module **telemetry** sets up the python enviroment and the simulated router test bed along with the base configurations of IPv4 addresses, OSPF configurations, MPLS configurations, Loopback interfaces, etc. After the import, the below code block initialises and brings up the emulator using **sim.start** API call.


In [1]:
from telemetry import *
sim = Vxr()
sim.no_image_copy=True

sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:v1.1.0 2021-04-23 09:07 output_dir:vxr.out
INFO:pyvxr.vxr:b10e8f6b8c14:/home/vxr/notebooks/Put-Technology-to-Work/Telemetry
INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/vykkmul6dk sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:rp1, rp3, rpe1, rpe3, ser1
INFO:pyvxr.console:rp3:wait for XR login prompt (console output captured in vxr.out/logs/console.rp3.log)
INFO:pyvxr.console:rpe1:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe1.log)
INFO:pyvxr.console:rpe3:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe3.log)
INFO:pyvxr.console:rp1:wait for XR login prompt (console output captured in vxr.out/logs/console.rp1.log)
INFO:pyvxr.console:rp3:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:rp3:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:rp3:login successful
INFO:pyvxr.bring

Sim launch failed (rp3:No IP address assigned to the management interface (MgmtEth0/RP0/CPU0/0). !!! CHECK vxr.out/logs/console.rp3.log FOR CLUES !!!)


>At this point, console access to the router is available. Optionally, you can access the simulated router consoles directly from your laptop  through ssh.

In [2]:
print('Consoles can be reached by:')
print('P1:', get_telnet_cmd(sim, 'rp1'), '\nP3:', get_telnet_cmd(sim, 'rp3'), '\nPE1:', get_telnet_cmd(sim, 'rpe1'), '\nPE3:', get_telnet_cmd(sim, 'rpe3'), '\nser1:', get_telnet_cmd(sim, 'ser1'))
print('or better:')
print('P1:', get_ssh_cmd(sim, 'rp1'), '\nP3:', get_ssh_cmd(sim, 'rp3'), '\nPE1:', get_ssh_cmd(sim, 'rpe1'),  '\nPE3:', get_ssh_cmd(sim, 'rpe3'), '\nser1:', get_ssh_cmd(sim, 'ser1')),
print('The password is cisco123')

Consoles can be reached by:
P1: telnet 172.17.0.2 44985 
P3: telnet 172.17.0.2 34380 
PE1: telnet 172.17.0.2 33912 
PE3: telnet 172.17.0.2 44277 
ser1: telnet 172.17.0.2 39404
or better:


KeyError: 'xr_redir22'

>Accessing the telnet consoles of all 4 routers from this notebook.

In [3]:
console_ports = sim.ports()
loginpe1 = telnetlib.Telnet(str(console_ports['rpe1']['HostAgent']) , str(console_ports['rpe1']['serial0']))
loginp1 = telnetlib.Telnet(str(console_ports['rp1']['HostAgent']) , str(console_ports['rp1']['serial0']))
loginp3 = telnetlib.Telnet(str(console_ports['rp3']['HostAgent']) , str(console_ports['rp3']['serial0']))
loginpe3 = telnetlib.Telnet(str(console_ports['rpe3']['HostAgent']) , str(console_ports['rpe3']['serial0']))
loginser1 = telnetlib.Telnet(str(console_ports['ser1']['HostAgent']) , str(console_ports['ser1']['serial0']))

## <a name="step3"></a>Configure a telemetry session between the router and the collector

Enter telemetry configuration mmode.

In [4]:
loginp1.write(('''
config
telemetry model-driven
!
commit
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',3)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#config
Wed Apr 21 11:56:31.209 UTC
RP/0/RP0/CPU0:P1(config)#telemetry model-driven
RP/0/RP0/CPU0:P1(config-model-driven)#!
RP/0/RP0/CPU0:P1(config-model-driven)#commit
Wed Apr 21 11:56:31.680 UTC
RP/0/RP0/CPU0:P1(config-model-driven)#


### <a name="step3a"></a>Create a Destination Group

Create one or more destinations to collect telemetry data from a router. Define a destination-group to contain the details about the destinations. Include the destination address (ipv4 or ipv6), port, transport, and encoding format in the destination-group.

> This use case uses 'self-describing-gpb' encoding and 'UDP' as the transport protocol. If you use TCP or gRPC, ensure that a receiver is setup at the collector. The receiver must match the values for encoding, transport and port that you set in the telemetry configuration.   

>Port number range <1-65535>.

> IMPORTANT: Do not forget to replace the destination address and port number of the collector in your setup. 

In [5]:
loginp1.write(('''
destination-group D1
   address-family ipv4 192.168.123.99 port 20030  
   encoding self-describing-gpb  
   protocol udp
 !
commit
exit
exit
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',3)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1(config-model-driven)#destination-group D1
RP/0/RP0/CPU0:P1(config-model-driven-dest)#   address-family ipv4 192.168.123.99 port 20030  
RP/0/RP0/CPU0:P1(config-model-driven-dest-addr)#   encoding self-describing-gpb  
RP/0/RP0/CPU0:P1(config-model-driven-dest-addr)#   protocol udp
RP/0/RP0/CPU0:P1(config-model-driven-dest-addr)# !
RP/0/RP0/CPU0:P1(config-model-driven-dest-addr)#commit
Wed Apr 21 11:56:36.964 UTC
RP/0/RP0/CPU0:P1(config-model-driven-dest-addr)#exit
RP/0/RP0/CPU0:P1(config-model-driven-dest)#exit
RP/0/RP0/CPU0:P1(config-model-driven)#


### <a name="step3b"></a>Create a Sensor-group

Specify the subset of the data that you want to stream from the router using sensor paths. The Sensor path represents the path in the hierarchy of a YANG data model. Create a sensor-group to contain the sensor paths.

>Here, the sensor path is for streaming CPU utilization.

In [6]:
loginp1.write(('''
sensor-group SGroup1
   sensor-path Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
 !  
commit
exit
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',3)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1(config-model-driven)#sensor-group SGroup1
RP/0/RP0/CPU0:P1(config-model-driven-snsr-grp)#   sensor-path Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
RP/0/RP0/CPU0:P1(config-model-driven-snsr-grp)# !  
RP/0/RP0/CPU0:P1(config-model-driven-snsr-grp)#commit
Wed Apr 21 11:56:40.829 UTC
RP/0/RP0/CPU0:P1(config-model-driven-snsr-grp)#exit
RP/0/RP0/CPU0:P1(config-model-driven)#


### <a name="step3c"></a>Create a Subscription

Subscribe to telemetry data that is streamed from a router. A Subscription binds the destination-group with the sensor-group and sets the streaming method. The streaming method can be Cadence-driven or Event-driven telemetry. Cadence-driven telemetry continually streams data (operational statistics and state transitions) at a configured cadence. Whereas, event-driven telemetry optimizes data that is collected at the receiver and streams data only when a state transition occurs.

In [7]:
loginp1.write(('''
subscription Sub1  
  sensor-group-id SGroup1 sample-interval 30000  
  destination-id D1 
 !
commit
root
exit
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',3)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

-interval 30000  
  destination-id D1 
 !
commit
root
exit
RP/0/RP0/CPU0:P1(config-model-driven)#subscription Sub1  
RP/0/RP0/CPU0:P1(config-model-driven-subs)#  sensor-group-id SGroup1 sample-interval 30000  
RP/0/RP0/CPU0:P1(config-model-driven-subs)#  destination-id D1 
RP/0/RP0/CPU0:P1(config-model-driven-subs)# !
RP/0/RP0/CPU0:P1(config-model-driven-subs)#commit
Wed Apr 21 11:56:43.855 UTC
RP/0/RP0/CPU0:P1(config-model-driven-subs)#root
RP/0/RP0/CPU0:P1(config)#exit
RP/0/RP0/CPU0:P1#


## <a name="step4"></a>Verify the Telemetry Configuration

Verify the deployment of the subscription on the router.

In [8]:
# Show the telemetry configuration applied on the router.
loginp1.write(('''
show running-config telemetry model-driven
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show running-config telemetry model-driven
Wed Apr 21 11:56:50.678 UTC
telemetry model-driven
 destination-group D1
  address-family ipv4 192.168.122.99 port 20030
   encoding self-describing-gpb
   protocol udp
  !
 !
 sensor-group SGroup1
  sensor-path Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
 !
 subscription Sub1
  sensor-group-id SGroup1 sample-interval 30000
  destination-id D1
 !
!

RP/0/RP0/CPU0:P1#


In [9]:
# Show the telemetry configuration applied on the router.
loginp1.write(('''
show telemetry model-driven subscription Sub1
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show telemetry model-driven subscription Sub1
Wed Apr 21 11:56:53.628 UTC
Subscription:  Sub1
-------------
  State:       NA
  Sensor groups:
  Id: SGroup1
    Sample Interval:      30000 ms
    Heartbeat Interval:   NA
    Sensor Path:          Cisco-IOS-XR-wdsysmon-fd-oper:system-monitoring/cpu-utilization
    Sensor Path State:    Not Resolved

  Destination Groups:
  Group Id: D1
    Destination IP:       192.168.122.99
    Destination Port:     20030
    Encoding:             self-describing-gpb
    Transport:            udp
    State:                NA
    TLS :                 False

  Collection Groups:
  ------------------
  No active collection groups

RP/0/RP0/CPU0:P1#


>After the telemetry session is established, the router streams data to the receiver to create a data lake. Check the collector console to see the streamed data.

## <a name="step5"></a>Operate on Telemetry Data

You can start consuming and analyzing telemetry data from the data lake using an open-sourced collection
stack based on your requirement. 
For example, the following tools can be used to analyze the collected telemetry data:
- Pipeline is a lightweight tool used to collect data. You can download Network Telemetry Pipeline from Github. You define how you want the collector to interact with routers and where you want to send the
processed data using pipeline.conf file.
- Telegraph (plugin-driven server agent) and InfluxDB (a time series database (TSDB)) stores telemetry
data, which is retrieved by visualization tools. You can download InfluxDB from Github. You define
what data you want to include into your TSDB using the metrics.json file.
- Grafana is a visualization tool that displays graphs and counters for data streamed from the router.

For more information, see http://xrdocs.io/telemetry/.

#### Clean-up the topology

>Once you are done with experimenting on the topology, you should bring down the emulator by executing the following steps:

In [2]:
# close our telnet
loginp1.close()
loginpe1.close()
loginser1.close()

NameError: name 'loginp1' is not defined

In [2]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/vykkmul6dk)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [ ]:
sim.clean()

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/vykkmul6dk)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [ ]:
# Clean up sim scratch space
shutil.rmtree(sim_dir)